<a href="https://colab.research.google.com/github/passivebook/FixWordPressLinks/blob/main/Fix_WordPress_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1T5JJmlDwdaEkAAbpH2VOn-TCeAuW8OKl

# Setup

In [1]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from bs4 import element as bs4_element
from google.colab import drive
import csv
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Set Google Drive Working Directory

Put the path to the google drive folder after `/content/gdrive/My Drive/`



In [2]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Fix WordPress Links")

## Upload the input.txt file to Google Drive 
1. Create a `input.txt` file and paste the WordPress Post Code in it. 
2. Upload the input.txt file to path setup above. 

## Check files


Check if files `input.txt` and `affiliate_links.csv` exists in Drive by running the code below. If you don't find the files, set the Google Drive working directory in the previous step. 

In [3]:
! ls

 affiliate_links.csv	 'Fix WordPress Links.ipynb'   output.txt
 affiliate_links.gsheet   input.txt


## Code Settings

In [4]:
filename = 'input.txt'
domain = 'passivebook.com'
affiliate_link = 'passivebook.com/go'

## Read the file

In [5]:
f = open(filename)
content = f.read()
#parse HTML
soup = BeautifulSoup(content, 'html.parser')
# Remove output.txt
try:
    os.remove('output.txt')
except OSError:
    pass

# Add Affiliate Links

In [20]:
html_doc = """

<p>You can use a tool like KeywordTool.io, Ubersuggest or Ahrefs to help you with this.</p>


<!-- wp:heading {"level":3} -->
<h3 id="usebuzzsumoahrefs">[auto-list-number] Use Buzzsumo &amp; Ahrefs</h3>
<!-- /wp:heading -->


<!-- wp:paragraph -->
<p>This is a existing link <a class="test" href="https://yahoo.com/">Yahoo</a> Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another Google Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another lowercase bing Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Active Campaign.</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Bunny CDN Text.</p>
<!-- /wp:paragraph -->


<!-- wp:paragraph -->
<p>This is another multi word Bunny Text</p>
<!-- /wp:paragraph -->
"""

# Uncomment for Testing
#soup = BeautifulSoup(html_doc, 'html.parser')


# Read & Sort the CSV File
hyperlinks = pd.read_csv('affiliate_links.csv', engine='python')
hyperlinks['anchor_length'] = hyperlinks['anchor_to_link'].str.len()
hyperlinks = hyperlinks.sort_values(['link_to_be_added', 'anchor_length'], ascending=[False,False]) 
del hyperlinks['anchor_length']
hyperlinks = dict(hyperlinks.values)


# read the CSV file with anchor text and hyperlinks
# with open('affiliate_links.csv', 'r') as csv_file:
#   reader = csv.reader(csv_file)
#   hyperlinks = dict(reader)


# from bs4 import element as bs4_element
be_navStr = bs4_element.NavigableString
 
hList = [
    (anchor_text.strip(), hyperlink.strip()) for
    anchor_text, hyperlink in hyperlinks.items()
    if anchor_text.strip() and hyperlink.strip() # no blanks
]
 
 
for txt, link in hList:

    navStrs = [
        d for d in soup.descendants if type(d) == be_navStr 
        # and f' {txt.lower()} ' in f' {d.string.strip().lower()} ' # same as
        # and f' {txt.lower()} ' in f' {d.string.strip().lower()} ' # same as
        # and (' '+txt.lower()+' ') in (' '+d.string.strip().lower()+' ')
        and (' '+re.sub('\W+',' ',txt.lower())+' ') in (' '+re.sub('\W+',' ',d.string.strip().lower())+' ')
    ]

    #if len(navStrs) > 0:
      #print(txt)
      #print(navStrs)

    for ns in navStrs: 
        # tLen, remStr = len(txt), f' {ns.get_text().strip()} '
        tLen, remStr = len(txt), f' {ns.string} '
        remLen = len(remStr)
        # tLen, remStr = len(txt), f' {ns.text.strip()} '

        if remStr[1:-1].lower() == txt.lower():
            # to skip if it's already a hyperlink
            if ns.parent.name == 'a': 
                ns.parent['href'] = link # comment if you dont want to replace/update link
                continue 
        # Skip creating nested hyperlinks inside existing hyperlinks       
        if ns.parent.name == 'a': 
          continue 




        while ' '+re.sub('\W+',' ',txt.lower())+' ' in re.sub('\W+',' ',remStr.lower()):
            
            #print(txt.lower())
            #print(re.sub('\W+',' ',remStr.lower()))

            sInd = remStr.lower().find(f'{txt.lower()}')
            #print(remStr.lower())
            #print(sInd)
            hlTag = soup.new_tag('a', href=link)
            hlTag.append(remStr[sInd:sInd + tLen])
            #print(hlTag)
            newCont = [remStr[1:sInd], hlTag]
            #print(newCont)

            for addn in newCont: ns.insert_before(addn)
            #print(soup)

            remStr = remStr[sInd + tLen:remLen-1]
            #print(remStr)

        ns.replace_with(remStr)
        #print(soup)

# Uncomment for Testing 
#print(soup)


<p>You can use a tool like <a href="https://passivebook.com/go/keywordtoolio/">KeywordTool.io</a>, <a href="https://passivebook.com/go/ubersuggest/">Ubersuggest</a> or <a href="https://passivebook.com/go/ahrefs/">Ahrefs</a> to help you with this.</p>
<!-- wp:heading {"level":3} -->
<h3 id="usebuzzsumoahrefs">[auto-list-number] Use <a href="https://passivebook.com/go/buzzsumo/">Buzzsumo</a> &amp; <a href="https://passivebook.com/go/ahrefs/">Ahrefs</a></h3>
<!-- /wp:heading -->
<!-- wp:paragraph -->
<p>This is a existing link <a class="test" href="https://yahoo.com/">Yahoo</a> Text</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>This is another Google Text</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>This is another lowercase bing Text</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>This is another multi word <a href="https://passivebook.com/go/activecampaign/">Active Campaign</a>.</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>This is another multi word <a href="htt

# Fix Links

In [8]:
html_doc = """
<!-- wp:heading -->
<h2>This is a & Heading 2</h2>
<!-- /wp:heading -->


<!-- wp:heading -->
<h3>This is a Heading 3?</h3>
<!-- /wp:heading -->


<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/go/trello"> Affiliate Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/calculators/blog-earning-calculator/" class="test">Internal Link </a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://google.com">External Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="#jumplink">Jump Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/go/TRELLO.md">Markdown</a></p>
<!-- /wp:paragraph -->


<!-- wp:paragraph -->
<p><a href="/go/Rank Math%20Pro">Link with space</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is a Google Text</p>
<!-- /wp:paragraph -->
"""

# Uncomment for Testing
#soup = BeautifulSoup(html_doc, 'html.parser')

In [9]:
# Operations on All Links
for link in soup.find_all('a'):
    # Remove .md Obsidian Markdown from links
    link['href'] = link['href'].replace(".md", "")
    link['href'] = link['href'].replace("app://obsidian", "")
    # Check if link is not an anchor link
    if "#" not in link['href']:
      # Set All Links to Open in New Tab
      link['target'] = '_blank'
      # Remove Leading and Trailiing White Spaces
      link.string = link.string.strip()
    # Add Domain to Internal Links
    if str(link['href'][0:1]) == "/":
      link['href'] = "https://" + domain + link['href'] 

# Internal Links
for link in soup.find_all('a', href=re.compile(domain)):
    # Convert all internal links to lower case
    link['href'] = link['href'].lower()        
    url = link['href']
    # Add Trailing / to all links
    if url[len(url)-1] != "/" and "#" not in link['href']:
        link['href'] = url + "/"
    # Replace space in internal links with -    
    link['href'] = link['href'].replace(" ", "-") 
    link['href'] = link['href'].replace("%20", "-")   

# Affiliate Links
for link in soup.find_all('a', href=re.compile(affiliate_link)):
    # Make Affiliate Links No Follow
    link['rel'] = 'nofollow'

# External Links
#for link in soup.find_all('a', href=re.compile(".*?:\\/\\/(?!"+domain+").*?")):
    #link['rel'] = link['rel'].remove('nofollow')
    #link['rel'] = 'nofollow'

# Uncomment for Testing
# print(soup)

# Add ID to Headings (h2 & h3)

In [10]:
# Add ID Tag to Headings: removes spaces, special characters and converts to lower case.
for heading in soup.find_all('h2'): heading['id'] = ''.join(e for e in heading.string if e.isalnum()).lower().replace(" ", "")
#for heading in soup.find_all('h3'): heading['id'] = ''.join(e for e in heading.string if e.isalnum()).lower().replace(" ", "")
#print(soup)

# Output File

In [11]:
with open("output.txt", "w", encoding='utf-8') as file:
    file.write(str(soup))

In [12]:
print(soup)

<!-- wp:paragraph -->
<p>Are you looking for blog post ideas? Are you struggling to come up with blog topics that will engage your readers and drive traffic to your blog?</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>If yes, you're not alone. Many bloggers face this challenge. But don't worry – help is here!</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>In this post, we'll provide blog post ideas to get you started. Using this list you will be able to come up with blog topics to write about that will boost both traffic and profits.</p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p>So read on – and start blogging!</p>
<!-- /wp:paragraph -->
<!-- wp:image {"id":17738,"sizeSlug":"large","linkDestination":"none"} -->
<figure class="wp-block-image size-large"><img alt="Blog Post Ideas With Examples FI" class="wp-image-17738" src="https://passivebook.com/wp-content/uploads/2022/12/Blog-Post-Ideas-With-Examples-FI-650x339.png"/></figure>
<!-- /wp:image -->
<!-- wp:heading -->
<h